In [4]:
import os
import yt_dlp
import numpy as np
import soundfile as sf
import pyloudnorm as pyln
from openpyxl import load_workbook
from pydub import AudioSegment
from mutagen.wave import WAVE

# === CONFIG ===
EXPORT_FOLDER = "lufs_audio"
TARGET_LUFS = -14.0
os.makedirs(EXPORT_FOLDER, exist_ok=True)

# Load song titles
def load_song_titles(xlsx_path):
    wb = load_workbook(xlsx_path)
    ws = wb.active
    return [row[0].value for row in ws.iter_rows(min_row=2) if row[0].value]

# Download highest quality audio
def download_best_audio(song_title):
    ydl_opts = {
        'format': 'bestaudio/best',
        'quiet': True,
        'default_search': 'ytsearch',
        'outtmpl': f'{EXPORT_FOLDER}/{song_title}.%(ext)s',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print(f"🔍 Downloading: {song_title}")
        try:
            ydl.download([song_title])
        except Exception as e:
            print(f"❌ Failed to download '{song_title}': {e}")

# Convert to WAV and apply LUFS normalization
def convert_to_lufs_wav(song_title):
    raw_path = None
    for ext in ["webm", "m4a", "opus", "mp3"]:
        candidate = os.path.join(EXPORT_FOLDER, f"{song_title}.{ext}")
        if os.path.exists(candidate):
            raw_path = candidate
            break

    if not raw_path:
        print(f"❌ No downloaded audio found for: {song_title}")
        return

    print(f"🎼 Converting and normalizing: {song_title}")
    audio = AudioSegment.from_file(raw_path)
    wav_temp_path = os.path.join(EXPORT_FOLDER, f"{song_title}_temp.wav")
    audio.export(wav_temp_path, format="wav")

    # Load raw WAV into numpy
    y, sr = sf.read(wav_temp_path)
    if len(y.shape) > 1:  # stereo to mono
        y = y.mean(axis=1)

    # Measure loudness
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(y)
    print(f"📏 Original LUFS: {loudness:.2f}")

    # Normalize
    y_normalized = pyln.normalize.loudness(y, loudness, TARGET_LUFS)

    # Export normalized audio
    final_wav_path = os.path.join(EXPORT_FOLDER, f"{song_title}.wav")
    sf.write(final_wav_path, y_normalized, sr)
    os.remove(wav_temp_path)
    os.remove(raw_path)
    print(f"✅ LUFS-normalized WAV saved: {final_wav_path}")

    # Optional metadata
    add_metadata(final_wav_path, song_title)

# Add metadata to WAV
def add_metadata(filepath, song_title):
    artist = "Unknown Artist"
    if " - " in song_title:
        artist, song_title = song_title.split(" - ", 1)

    try:
        audio = WAVE(filepath)
        audio["title"] = song_title
        audio["artist"] = artist
        audio.save()
        print(f"🏷️ Metadata tagged: {song_title} by {artist}")
    except Exception as e:
        print(f"⚠️ Metadata tagging failed: {e}")

# Main
if __name__ == "__main__":
    titles = load_song_titles("songs.xlsx")
    for title in titles:
        download_best_audio(title)
        convert_to_lufs_wav(title)


🔍 Downloading: Marty Mcfly - The Secret Garden
🎼 Converting and normalizing: Marty Mcfly - The Secret Garden
📏 Original LUFS: -17.35


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly - The Secret Garden.wav
⚠️ Metadata tagging failed: 'The Secret Garden' not a Frame instance
🔍 Downloading: Marty Mcfly & Adam Holtz - Please Come Back
🎼 Converting and normalizing: Marty Mcfly & Adam Holtz - Please Come Back
📏 Original LUFS: -12.62
✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly & Adam Holtz - Please Come Back.wav
⚠️ Metadata tagging failed: 'Please Come Back' not a Frame instance
🔍 Downloading: Marty Mcfly ft Ria & Cookie - Diane
🎼 Converting and normalizing: Marty Mcfly ft Ria & Cookie - Diane
📏 Original LUFS: -18.51
✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly ft Ria & Cookie - Diane.wav
⚠️ Metadata tagging failed: 'Diane' not a Frame instance


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


🔍 Downloading: Marty Mcfly & Adam Holtz - That's When Love Dies
🎼 Converting and normalizing: Marty Mcfly & Adam Holtz - That's When Love Dies
📏 Original LUFS: -15.53


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly & Adam Holtz - That's When Love Dies.wav
⚠️ Metadata tagging failed: "That's When Love Dies" not a Frame instance
🔍 Downloading: Marty Mcfly & Adam Holtz - Love Drives You Insane
🎼 Converting and normalizing: Marty Mcfly & Adam Holtz - Love Drives You Insane
📏 Original LUFS: -12.70
✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly & Adam Holtz - Love Drives You Insane.wav
⚠️ Metadata tagging failed: 'Love Drives You Insane' not a Frame instance
🔍 Downloading: Marty Mcfly and Adam Holtz - Smile when I'm gone
🎼 Converting and normalizing: Marty Mcfly and Adam Holtz - Smile when I'm gone
📏 Original LUFS: -14.16


c:\Users\PX\anaconda3\envs\yt_autotune\lib\site-packages\pyloudnorm\normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly and Adam Holtz - Smile when I'm gone.wav
⚠️ Metadata tagging failed: "Smile when I'm gone" not a Frame instance
🔍 Downloading: Marty Mcfly & Adam Holtz - Why Did You Leave Me
🎼 Converting and normalizing: Marty Mcfly & Adam Holtz - Why Did You Leave Me
📏 Original LUFS: -12.59
✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly & Adam Holtz - Why Did You Leave Me.wav
⚠️ Metadata tagging failed: 'Why Did You Leave Me' not a Frame instance
🔍 Downloading: Marty Mcfly & Adam Holtz - Why Did You Go
🎼 Converting and normalizing: Marty Mcfly & Adam Holtz - Why Did You Go
📏 Original LUFS: -12.17
✅ LUFS-normalized WAV saved: lufs_audio\Marty Mcfly & Adam Holtz - Why Did You Go.wav
⚠️ Metadata tagging failed: 'Why Did You Go' not a Frame instance
